In this notebook we will consolidate all the variety of data sources into a single dataset, as well as do the ad hoc data cleaning that the process requires. Before diving in, check out the read me/cover page to get some background on the project. 

First lets import our libraries and the first data source we will massage: suicide data (broken down by county) from the CDC. 

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
df_suicides = pd.read_csv('Suicides.csv')
df_suicides

,Notes,County,County Code,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate
0,NaN,"Autauga County, AL",1001.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,115.0,609875.0,18.9
1,NaN,"Autauga County, AL",1001.0,Intentional self-harm (suicide) by discharge o...,GR113-125,78.0,609875.0,12.8
2,NaN,"Autauga County, AL",1001.0,Intentional self-harm (suicide) by other and u...,GR113-126,37.0,609875.0,6.1
3,NaN,"Baldwin County, AL",1003.0,"#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,444.0,2250866.0,19.7
4,NaN,"Baldwin County, AL",1003.0,Intentional self-harm (suicide) by discharge o...,GR113-125,281.0,2250866.0,12.5
...,...,...,...,...,...,...,...,...
7637,resident population that is under one year of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7638,"15. Beginning with the 2018 data, changes have...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7639,"compared to previous practice. In addition, da...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7640,in information available to individual states ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here can see what the data is actually structured like (using the head function), the type of death is split into seperate rows. We can also see from the isnull function that we are seeing 7544 rows of data and 98 rows of 'notes' that we will remove.

Below we can see the different cause lists that indicate what the death value actually repersents. We can see there is a row that is the Total suicides, and the other 2 rows indicates whether it was via firearm or another method. 

In [94]:
df_suicides['ICD-10 113 Cause List'].value_counts()
#check the different values of the cause list

#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)                                                             2813
Intentional self-harm (suicide) by discharge of firearms (X72-X74)                                                2585
Intentional self-harm (suicide) by other and unspecified means and their sequelae (*U03,X60-X71,X75-X84,Y87.0)    2146
Name: ICD-10 113 Cause List, dtype: int64

In [95]:
(df_suicides.isnull().sum(axis=0))
#check the null values of each column

Notes                         7544
County                          98
County Code                     98
ICD-10 113 Cause List           98
ICD-10 113 Cause List Code      98
Deaths                          98
Population                      98
Crude Rate                      98
dtype: int64

In [96]:
df_suicides = df_suicides.dropna(subset = ['County'])
#drop the NaN in the county column, to drop the 'Notes' rows

In [97]:
(df_suicides.isnull().sum(axis=0))
#Now we can see that we have no null in our data columns

Notes                         7544
County                           0
County Code                      0
ICD-10 113 Cause List            0
ICD-10 113 Cause List Code       0
Deaths                           0
Population                       0
Crude Rate                       0
dtype: int64

In [98]:
df_suicides = df_suicides[['County','ICD-10 113 Cause List','ICD-10 113 Cause List Code','Deaths','Population']]
df_suicides
#Limit our tables to the rows we are actually interested in 

,County,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population
0,"Autauga County, AL","#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,115.0,609875.0
1,"Autauga County, AL",Intentional self-harm (suicide) by discharge o...,GR113-125,78.0,609875.0
2,"Autauga County, AL",Intentional self-harm (suicide) by other and u...,GR113-126,37.0,609875.0
3,"Baldwin County, AL","#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,444.0,2250866.0
4,"Baldwin County, AL",Intentional self-harm (suicide) by discharge o...,GR113-125,281.0,2250866.0
...,...,...,...,...,...
7539,"Uinta County, WY",Intentional self-harm (suicide) by other and u...,GR113-126,19.0,227928.0
7540,"Washakie County, WY","#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,14.0,90346.0
7541,"Washakie County, WY",Intentional self-harm (suicide) by discharge o...,GR113-125,10.0,90346.0
7542,"Weston County, WY","#Intentional self-harm (suicide) (*U03,X60-X84...",GR113-124,19.0,77791.0


Since we are using suicide rates as a proxy of gun ownership rate we can exclude rows for non-firearm suicides. Once I did that, I decided to create a new column ('Suicide Cate') to map the translate Cause List Code to simpler name. Finally, we will store the data in a pivot to ensure that each row only contains data for a single county. 

In [99]:
df_suicides = df_suicides.loc[df_suicides['ICD-10 113 Cause List Code'] != 'GR113-126']
#drop the code which include non-fireman suicides

In [100]:
df_suicides['ICD-10 113 Cause List'].value_counts()
#look at the update value counts

#Intentional self-harm (suicide) (*U03,X60-X84,Y87.0)                 2813
Intentional self-harm (suicide) by discharge of firearms (X72-X74)    2585
Name: ICD-10 113 Cause List, dtype: int64

In [101]:
df_suicides['Suicide Cate'] = ''
df_suicides.loc[df_suicides['ICD-10 113 Cause List Code']== 'GR113-125','Suicide Cate'] = 'Firearm Suicides'
df_suicides.loc[df_suicides['ICD-10 113 Cause List Code']== 'GR113-124','Suicide Cate'] = 'Total Suicides'
#create a new column with more user friendly values

C:\Users\rites\AppData\Local\Temp\ipykernel_20364\423370220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suicides['Suicide Cate'] = ''


In [102]:
df_suicides = df_suicides[['County','Suicide Cate','Deaths']]
#cut our table down to include only the neccesary fields

In [103]:
df_suicides_pivot = df_suicides.reset_index().pivot(index = 'County', columns = 'Suicide Cate', values = 'Deaths').reset_index()
df_suicides_pivot

#create the pivot table

Suicide Cate,County,Firearm Suicides,Total Suicides
0,"Abbeville County, SC",43.0,51.0
1,"Acadia Parish, LA",72.0,115.0
2,"Accomack County, VA",31.0,55.0
3,"Ada County, ID",496.0,872.0
4,"Adair County, IA",NaN,16.0
...,...,...,...
2808,"Yukon-Koyukuk Census Area, AK",19.0,33.0
2809,"Yuma County, AZ",175.0,278.0
2810,"Yuma County, CO",NaN,12.0
2811,"Zapata County, TX",NaN,10.0


Now we are done with our Suicide Table, we will clean our Homicides data. These values are actually from the same source as the suicide table so the data cleaning process is very similiar. 

In [104]:
df_homicides = pd.read_csv('Homicide.csv')
df_homicides
#import data

,Notes,County,County Code,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate
0,NaN,"Autauga County, AL",1001.0,"#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,32.0,609875.0,5.2
1,NaN,"Autauga County, AL",1001.0,Assault (homicide) by discharge of firearms (*...,GR113-128,22.0,609875.0,3.6
2,NaN,"Autauga County, AL",1001.0,Assault (homicide) by other and unspecified me...,GR113-129,10.0,609875.0,Unreliable
3,NaN,"Baldwin County, AL",1003.0,"#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,68.0,2250866.0,3.0
4,NaN,"Baldwin County, AL",1003.0,Assault (homicide) by discharge of firearms (*...,GR113-128,49.0,2250866.0,2.2
...,...,...,...,...,...,...,...,...
3952,"4, 2021 to include corrected information for r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,Terrorism involving firearms (homicide). The u...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,2019 data. Underlying and multiple cause of de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,"unspecified firearm discharge, ICD-10 code X95...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
(df_homicides.isnull().sum(axis=0))
#check nulls

Notes                         3853
County                         104
County Code                    104
ICD-10 113 Cause List          104
ICD-10 113 Cause List Code     104
Deaths                         104
Population                     104
Crude Rate                     104
dtype: int64

In [106]:
df_homicides = df_homicides.dropna(subset = ['County'])
#drop the nan 

In [107]:
df_homicides = df_homicides[['County','ICD-10 113 Cause List','ICD-10 113 Cause List Code','Deaths','Population']]
df_homicides

,County,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population
0,"Autauga County, AL","#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,32.0,609875.0
1,"Autauga County, AL",Assault (homicide) by discharge of firearms (*...,GR113-128,22.0,609875.0
2,"Autauga County, AL",Assault (homicide) by other and unspecified me...,GR113-129,10.0,609875.0
3,"Baldwin County, AL","#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,68.0,2250866.0
4,"Baldwin County, AL",Assault (homicide) by discharge of firearms (*...,GR113-128,49.0,2250866.0
...,...,...,...,...,...
3848,"Natrona County, WY","#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,43.0,875586.0
3849,"Natrona County, WY",Assault (homicide) by discharge of firearms (*...,GR113-128,29.0,875586.0
3850,"Natrona County, WY",Assault (homicide) by other and unspecified me...,GR113-129,14.0,875586.0
3851,"Sheridan County, WY","#Assault (homicide) (*U01-*U02,X85-Y09,Y87.1)",GR113-127,11.0,329807.0


In [108]:
df_homicides = df_homicides.loc[df_homicides['ICD-10 113 Cause List Code'] != 'GR113-129']
#drop rows of non-firearm homicides

In [109]:
df_homicides['Homicide Cate'] = ''
df_homicides.loc[df_homicides['ICD-10 113 Cause List Code']== 'GR113-128','Homicide Cate'] = 'Firearm Homicide'
df_homicides.loc[df_homicides['ICD-10 113 Cause List Code']== 'GR113-127','Homicide Cate'] = 'Total Homicide'
#create a new column with more user friendly values 

C:\Users\rites\AppData\Local\Temp\ipykernel_20364\532053354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homicides['Homicide Cate'] = ''


In [110]:
df_homicides = df_homicides[['County','Homicide Cate','Deaths','Population']]
df_homicides

,County,Homicide Cate,Deaths,Population
0,"Autauga County, AL",Total Homicide,32.0,609875.0
1,"Autauga County, AL",Firearm Homicide,22.0,609875.0
3,"Baldwin County, AL",Total Homicide,68.0,2250866.0
4,"Baldwin County, AL",Firearm Homicide,49.0,2250866.0
6,"Barbour County, AL",Total Homicide,29.0,287620.0
...,...,...,...,...
3846,"Laramie County, WY",Firearm Homicide,27.0,1063754.0
3848,"Natrona County, WY",Total Homicide,43.0,875586.0
3849,"Natrona County, WY",Firearm Homicide,29.0,875586.0
3851,"Sheridan County, WY",Total Homicide,11.0,329807.0


In [111]:
df_homicides_pivot = df_homicides.reset_index().pivot(index = ['County','Population'], columns = 'Homicide Cate', values = 'Deaths')
df_homicides_pivot = df_homicides_pivot.reset_index()
#create the pivot table for homicides 

In [112]:
df_homicides_pivot

Homicide Cate,County,Population,Firearm Homicide,Total Homicide
0,"Abbeville County, SC",273649.0,13.0,22.0
1,"Acadia Parish, LA",684322.0,59.0,72.0
2,"Accomack County, VA",361441.0,33.0,43.0
3,"Ada County, ID",4826008.0,37.0,68.0
4,"Adair County, OK",244203.0,12.0,22.0
...,...,...,...,...
1681,"York County, SC",2796307.0,103.0,131.0
1682,"York County, VA",739232.0,15.0,22.0
1683,"Yuba County, CA",828632.0,31.0,52.0
1684,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0


In [113]:
df = df_homicides_pivot.merge(df_suicides_pivot, how = 'left', on = 'County')

Finally we will replace some old county names with the updated county names to be compatible with the other datasets. 

In [114]:
df = df.replace({'LaSalle County, IL': 'La Salle County, IL','Shannon County, SD': 'Oglala Lakota County, SD',
                        'Wade Hampton Census Area, AK' : 'Kusilvak Census Area, AK'})

df

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides
0,"Abbeville County, SC",273649.0,13.0,22.0,43.0,51.0
1,"Acadia Parish, LA",684322.0,59.0,72.0,72.0,115.0
2,"Accomack County, VA",361441.0,33.0,43.0,31.0,55.0
3,"Ada County, ID",4826008.0,37.0,68.0,496.0,872.0
4,"Adair County, OK",244203.0,12.0,22.0,18.0,36.0
...,...,...,...,...,...,...
1681,"York County, SC",2796307.0,103.0,131.0,265.0,426.0
1682,"York County, VA",739232.0,15.0,22.0,52.0,95.0
1683,"Yuba County, CA",828632.0,31.0,52.0,72.0,143.0
1684,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0,19.0,33.0


Finally we can export the data into another csv. Now we can continue in the process, we can bring in the Income/ Poverty data.

In [115]:
df.to_csv('County_CDC_data.csv', index = False)

In [116]:
df = pd.read_csv('County_CDC_data.csv')

df_income = pd.read_csv('Income_data_expanded.csv')
df

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides
0,"Abbeville County, SC",273649.0,13.0,22.0,43.0,51.0
1,"Acadia Parish, LA",684322.0,59.0,72.0,72.0,115.0
2,"Accomack County, VA",361441.0,33.0,43.0,31.0,55.0
3,"Ada County, ID",4826008.0,37.0,68.0,496.0,872.0
4,"Adair County, OK",244203.0,12.0,22.0,18.0,36.0
...,...,...,...,...,...,...
1681,"York County, SC",2796307.0,103.0,131.0,265.0,426.0
1682,"York County, VA",739232.0,15.0,22.0,52.0,95.0
1683,"Yuba County, CA",828632.0,31.0,52.0,72.0,143.0
1684,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0,19.0,33.0


Unlike the CDC datasets, this dataset (from the census.gov) the data is much more cleaner and readable. So to merge the datasets all we need to do is create a column with the consildated county name. 

In [117]:
df_income['County'] = df_income['Name'] + ', ' + df_income['Postal Code']
df_income
#create the county column

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages","Poverty Percent, Age 0-17",Median Household Income,County
0,0,0,US,United States,12.8,16.9,"69,717","United States, US"
1,1,0,AL,Alabama,16.3,22.7,"53,990","Alabama, AL"
2,1,1,AL,Autauga County,10.7,16.1,"66,444","Autauga County, AL"
3,1,3,AL,Baldwin County,10.8,16.4,"65,658","Baldwin County, AL"
4,1,5,AL,Barbour County,23.0,35.1,"38,649","Barbour County, AL"
...,...,...,...,...,...,...,...,...
3190,56,37,WY,Sweetwater County,9.2,9.7,"74,677","Sweetwater County, WY"
3191,56,39,WY,Teton County,5.9,5.5,"102,709","Teton County, WY"
3192,56,41,WY,Uinta County,9.4,11.2,"70,162","Uinta County, WY"
3193,56,43,WY,Washakie County,10.2,12.5,"62,176","Washakie County, WY"


In [118]:
df_comb = df.merge(df_income, how = 'left', on = 'County')
df_comb
#merge the datasets together 

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages","Poverty Percent, Age 0-17",Median Household Income
0,"Abbeville County, SC",273649.0,13.0,22.0,43.0,51.0,45,1,SC,Abbeville County,14.8,21.2,"49,485"
1,"Acadia Parish, LA",684322.0,59.0,72.0,72.0,115.0,22,1,LA,Acadia Parish,20.6,28.7,"44,648"
2,"Accomack County, VA",361441.0,33.0,43.0,31.0,55.0,51,1,VA,Accomack County,16.2,25.7,"50,949"
3,"Ada County, ID",4826008.0,37.0,68.0,496.0,872.0,16,1,ID,Ada County,8.7,9.1,"79,345"
4,"Adair County, OK",244203.0,12.0,22.0,18.0,36.0,40,1,OK,Adair County,20.8,28.5,"43,371"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,"York County, SC",2796307.0,103.0,131.0,265.0,426.0,45,91,SC,York County,9.7,12.4,"73,466"
1683,"York County, VA",739232.0,15.0,22.0,52.0,95.0,51,199,VA,York County,5.1,6.1,"97,500"
1684,"Yuba County, CA",828632.0,31.0,52.0,72.0,143.0,6,115,CA,Yuba County,15.6,21.9,"60,764"
1685,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0,19.0,33.0,2,290,AK,Yukon-Koyukuk Census Area,23.7,32.1,"44,934"


Next we can check to see that each county only appears once in the dataset. As we can see, for some reason District of Colubmia is coming twice. After making sure both rows are actually the same I drop the duplicates.

In [119]:
df_comb['County'].value_counts()

District of Columbia, DC     2
Abbeville County, SC         1
Northumberland County, PA    1
Okaloosa County, FL          1
Ohio County, KY              1
                            ..
Giles County, TN             1
Gilchrist County, FL         1
Gila County, AZ              1
Gibson County, TN            1
Yuma County, AZ              1
Name: County, Length: 1686, dtype: int64

In [120]:
df_comb.loc[df_comb['County']== 'District of Columbia, DC']

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages","Poverty Percent, Age 0-17",Median Household Income
432,"District of Columbia, DC",7325774.0,1038.0,1344.0,104.0,462.0,11,0,DC,District of Columbia,16.8,25.5,"91,072"
433,"District of Columbia, DC",7325774.0,1038.0,1344.0,104.0,462.0,11,1,DC,District of Columbia,16.8,25.5,"91,072"


In [122]:
df_comb = df_comb[['County','Population','Firearm Homicide','Total Homicide', 'Firearm Suicides','Total Suicides',
         'Poverty Percent, All Ages','Poverty Percent, Age 0-17','Median Household Income']].drop_duplicates()

Finally, we can bring our final dataset, which has the population densities of each county. Before merging the data together, we need to change the states abbreviations to the full states name. Then we combine the state with the county to create the shared column the merge will occur on.  

In [123]:
df_pop_den = pd.read_csv('Average_Household_Size_and_Population_Density_-_County.csv')
df_pop_den
#import population density data

,OBJECTID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,...,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
0,1,161526,1001,1.539602e+09,25706961,Autauga County,Alabama,2.59,0.05,2.59,...,55200,NaN,35.853419,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.066037,0.150256
1,2,161527,1003,4.117547e+09,1133055836,Baldwin County,Alabama,2.61,0.04,2.66,...,208107,NaN,50.541504,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,4.483746,0.409904
2,3,161528,1005,2.292145e+09,50538698,Barbour County,Alabama,2.49,0.07,2.44,...,25782,NaN,11.247981,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.695262,0.223270
3,4,161529,1007,1.612167e+09,9602089,Bibb County,Alabama,2.99,0.14,3.05,...,22527,NaN,13.973114,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,1.887514,0.156473
4,5,161530,1009,1.670104e+09,15015423,Blount County,Alabama,2.77,0.05,2.85,...,57645,NaN,34.515816,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.423552,0.164405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,3216,1804553,72145,1.187776e+08,57795019,Vega Baja Municipio,Puerto Rico,2.93,0.09,2.93,...,53371,NaN,449.335380,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.485650,0.010381
3216,3217,1804554,72147,1.315414e+08,552192819,Vieques Municipio,Puerto Rico,3.55,0.38,3.03,...,8771,NaN,66.678630,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.675819,0.012181
3217,3218,1804555,72149,9.229857e+07,3622639,Villalba Municipio,Puerto Rico,2.96,0.10,3.09,...,22993,NaN,249.115455,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.431598,0.008189
3218,3219,1804556,72151,1.430052e+08,72592521,Yabucoa Municipio,Puerto Rico,2.91,0.10,2.90,...,34149,NaN,238.795547,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.676628,0.012229


In [124]:
df_pop_den = df_pop_den[['NAME','State','B01001_calc_PopDensity']]
#remove unused columns

In [125]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

df_pop_den['St'] = df_pop_den['State'].replace(us_state_to_abbrev)
#change state abbreviations to full state name

C:\Users\rites\AppData\Local\Temp\ipykernel_20364\627980589.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_den['St'] = df_pop_den['State'].replace(us_state_to_abbrev)


In [126]:
df_pop_den['County'] = df_pop_den['NAME'] + ', ' + df_pop_den['St']
df_pop_den
#create the combined county column 

C:\Users\rites\AppData\Local\Temp\ipykernel_20364\1055770183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_den['County'] = df_pop_den['NAME'] + ', ' + df_pop_den['St']


,NAME,State,B01001_calc_PopDensity,St,County
0,Autauga County,Alabama,35.853419,AL,"Autauga County, AL"
1,Baldwin County,Alabama,50.541504,AL,"Baldwin County, AL"
2,Barbour County,Alabama,11.247981,AL,"Barbour County, AL"
3,Bibb County,Alabama,13.973114,AL,"Bibb County, AL"
4,Blount County,Alabama,34.515816,AL,"Blount County, AL"
...,...,...,...,...,...
3215,Vega Baja Municipio,Puerto Rico,449.335380,PR,"Vega Baja Municipio, PR"
3216,Vieques Municipio,Puerto Rico,66.678630,PR,"Vieques Municipio, PR"
3217,Villalba Municipio,Puerto Rico,249.115455,PR,"Villalba Municipio, PR"
3218,Yabucoa Municipio,Puerto Rico,238.795547,PR,"Yabucoa Municipio, PR"


In [127]:
df_pop_den = df_pop_den[['County','B01001_calc_PopDensity']]
df_pop_den

,County,B01001_calc_PopDensity
0,"Autauga County, AL",35.853419
1,"Baldwin County, AL",50.541504
2,"Barbour County, AL",11.247981
3,"Bibb County, AL",13.973114
4,"Blount County, AL",34.515816
...,...,...
3215,"Vega Baja Municipio, PR",449.335380
3216,"Vieques Municipio, PR",66.678630
3217,"Villalba Municipio, PR",249.115455
3218,"Yabucoa Municipio, PR",238.795547


In [128]:
df_unadj = df_comb.merge(df_pop_den, how = 'left', on = 'County')

df_unadj

#merge the data 

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides,"Poverty Percent, All Ages","Poverty Percent, Age 0-17",Median Household Income,B01001_calc_PopDensity
0,"Abbeville County, SC",273649.0,13.0,22.0,43.0,51.0,14.8,21.2,"49,485",19.372405
1,"Acadia Parish, LA",684322.0,59.0,72.0,72.0,115.0,20.6,28.7,"44,648",36.872362
2,"Accomack County, VA",361441.0,33.0,43.0,31.0,55.0,16.2,25.7,"50,949",28.135968
3,"Ada County, ID",4826008.0,37.0,68.0,496.0,872.0,8.7,9.1,"79,345",163.707537
4,"Adair County, OK",244203.0,12.0,22.0,18.0,36.0,20.8,28.5,"43,371",14.882667
...,...,...,...,...,...,...,...,...,...,...
1681,"York County, SC",2796307.0,103.0,131.0,265.0,426.0,9.7,12.4,"73,466",146.710521
1682,"York County, VA",739232.0,15.0,22.0,52.0,95.0,5.1,6.1,"97,500",249.474195
1683,"Yuba County, CA",828632.0,31.0,52.0,72.0,143.0,15.6,21.9,"60,764",46.119104
1684,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0,19.0,33.0,23.7,32.1,"44,934",0.014362


Finally, before exporting our data to a csv we will change some of the column names into cleaner values.

In [129]:
df_unadj = df_unadj.rename(columns = {'Poverty Percent, All Ages' : 'Poverty %', 'Poverty Percent, Age 0-17' : 'Poverty % (U18)'
                                      ,'B01001_calc_PopDensity' : 'PopDen', 'Median Household Income' : 'MedHHInc'})

df_unadj

,County,Population,Firearm Homicide,Total Homicide,Firearm Suicides,Total Suicides,Poverty %,Poverty % (U18),MedHHInc,PopDen
0,"Abbeville County, SC",273649.0,13.0,22.0,43.0,51.0,14.8,21.2,"49,485",19.372405
1,"Acadia Parish, LA",684322.0,59.0,72.0,72.0,115.0,20.6,28.7,"44,648",36.872362
2,"Accomack County, VA",361441.0,33.0,43.0,31.0,55.0,16.2,25.7,"50,949",28.135968
3,"Ada County, ID",4826008.0,37.0,68.0,496.0,872.0,8.7,9.1,"79,345",163.707537
4,"Adair County, OK",244203.0,12.0,22.0,18.0,36.0,20.8,28.5,"43,371",14.882667
...,...,...,...,...,...,...,...,...,...,...
1681,"York County, SC",2796307.0,103.0,131.0,265.0,426.0,9.7,12.4,"73,466",146.710521
1682,"York County, VA",739232.0,15.0,22.0,52.0,95.0,5.1,6.1,"97,500",249.474195
1683,"Yuba County, CA",828632.0,31.0,52.0,72.0,143.0,15.6,21.9,"60,764",46.119104
1684,"Yukon-Koyukuk Census Area, AK",60314.0,NaN,14.0,19.0,33.0,23.7,32.1,"44,934",0.014362


In [130]:
df_unadj.to_csv('final_dataset.csv', index = False)

#END HERE

Now we are done with the data modification process and have our data in a single data file. Continue to part 2 for the EDA process. 